# Import

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [ ]:
from Saves.HelperFunctions import *
from Preprocessing import preprocess

In [ ]:
train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv('Data/test.csv')
# test_data = None

In [ ]:
train_data.head(2)

In [ ]:
# test_data.head(3)

# Preprocessing

In [ ]:
pre_method = 2
X_train_padded, Y_train, X_test_padded, num_unique_words, max_sequence_length = preprocess(train_data=train_data,
                                                                                            test_data=test_data, 
                                                                                            pre_method=pre_method,
                                                                                            fx_opt=2)

In [ ]:
print(f"X_train.shape: {X_train_padded.shape}")
print(f"X_test.shape: {X_test_padded.shape}")
print(f"Y_train.shape: {Y_train.shape}")

In [ ]:
Y_train.head(3)

# RNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

In [ ]:
print(f"num_unique_words: {num_unique_words}")
print(f"max_sequence_length: {max_sequence_length}")

In [ ]:
num_classes = 5
# Define the model
rnn_model = Sequential()

# Embedding layer to convert word indices into dense vectors of fixed size
rnn_model.add(Embedding(input_dim=num_unique_words, output_dim=128, input_length=max_sequence_length))

# RNN layer (SimpleRNN)
rnn_model.add(SimpleRNN(units=64, return_sequences=False))

# Dropout for regularization
rnn_model.add(Dropout(0.5))

# Fully connected layer for classification
rnn_model.add(Dense(5, activation='softmax'))  # Adjust output units for the number of classes

# Explicitly build the model
rnn_model.build(input_shape=(None, max_sequence_length))

# Compile the model
rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
rnn_model.summary()

In [ ]:
# Assuming Y_train contains labels in integer form (e.g., [0, 1, 2, 3, 4])
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)

In [ ]:
epochs = 5
batch_size = 32
validation_split = 0.2

In [ ]:
rnn_model.fit(X_train_padded, Y_train_categorical, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

## Evaluation

### Train

In [ ]:
# from tensorflow.keras.models import load_model
# model_loaded = load_model('Models/RNN/RNN-m2-e5-a83.h5') if os.path.exists('Models/RNN/RNN-m2-e5-a83.h5') else model

In [ ]:
# Predict the labels for the test set
predictions = rnn_model.predict(X_train_padded)

# If it's a multi-class classification task, get the predicted class for each sample
Y_train_pred = np.argmax(predictions, axis=1)

In [ ]:
train_accuracy = accuracy_score(Y_train_pred, Y_train)
print(f"Train Accuracy: {train_accuracy}")

In [ ]:
# file_name = f'RNN-m{pre_method}-e{epochs}-a{int(train_accuracy*100)}'
# rnn_model.save(f'Models/RNN/{file_name}.h5')

### Test

In [ ]:
# # Predict the labels for the test set
# predictions = rnn_model.predict(X_test_padded)

# # If it's a multi-class classification task, get the predicted class for each sample
# Y_test_pred = np.argmax(predictions, axis=1)

In [ ]:
# file_name = f'FFNN-m{pre_method}-e{epochs}'
# save_csv(data=Y_test_pred, file_name=file_name, header=['SampleID', 'Category'], numbering=True)